# Тестирование моделей

## Проверка эндпоинтов выделения признаков

In [1]:
import requests
import json
import numpy as np
import pandas as pd

API_URL = "http://localhost:8000/features/extract"

df = pd.read_csv("data/time_series/current_1.csv")
data = {
    "current_R": df.iloc[:50000, 0].tolist(),
    "current_S": df.iloc[:50000, 1].tolist(),
    "current_T": df.iloc[:50000, 2].tolist()
}

print("1. Real data windowing")
r1 = requests.post(API_URL, data={
    "raw_data": json.dumps(data),
    "user_id": "real_user", 
    "save_results": True,
    "use_windowing": True
})
if r1.status_code == 200:
    result1 = r1.json()
    extraction_id = result1["extraction_id"]
    print(f"ID: {extraction_id}, Windows: {result1['features']['total_windows']}")
else:
    print(f"Error: {r1.status_code}")

print("\n2. Random data single")
random_data = {k: np.random.randn(16384).tolist() for k in data.keys()}
r2 = requests.post(API_URL, data={
    "raw_data": json.dumps(random_data),
    "user_id": "test_user",
    "save_results": True
})
if r2.status_code == 200:
    result2 = r2.json()
    feature_count = len([k for group in result2['features'].values() if isinstance(group, dict) for k in group.keys()])
    print(f"ID: {result2['extraction_id']}, Features: {feature_count}")
else:
    print(f"Error: {r2.status_code}")

print("\n3. Get saved")
r3 = requests.get(f"http://localhost:8000/features/extract/{extraction_id}")
if r3.status_code == 200:
    print(f"Status: {r3.status_code}, Windows: {r3.json()['total_windows']}")
else:
    print(f"Error: {r3.status_code}")

print("\n4. User history")
r4 = requests.get("http://localhost:8000/features/history/real_user")
if r4.status_code == 200:
    history = r4.json()['extractions']
    print(f"Extractions: {len(history)}")
    for i, ext in enumerate(history[:2]):
        print(f"  {i+1}: windowing={ext['metadata']['use_windowing']}, features={ext['feature_count']}")
else:
    print(f"Error: {r4.status_code}")

print("\n5. CSV output")
r5 = requests.post(API_URL, data={
    "raw_data": json.dumps({k: v[:16384] for k, v in data.items()}),
    "output_format": "csv",
    "save_results": False
})
if r5.status_code == 200:
    csv_lines = r5.text.split('\n')
    headers = csv_lines[0].split(',') if csv_lines else []
    print(f"CSV: {len(csv_lines)} lines, {len(headers)} columns")
else:
    print(f"Error: {r5.status_code}")


1. Real data windowing
ID: 5603b396-b009-4a3f-9310-22b4c23f7031, Windows: 9

2. Random data single
ID: c92830b2-93e6-43a0-9c4c-3d8039ff1af9, Features: 119

3. Get saved
Status: 200, Windows: 9

4. User history
Extractions: 9
  1: windowing=True, features=119
  2: windowing=True, features=119

5. CSV output
CSV: 3 lines, 5 columns


In [2]:
print("\n3. Get saved")

extraction_id = '0a2fd7fe-58bb-467a-ac75-e30d9c1032bf'

r3 = requests.get(f"http://localhost:8000/features/extract/{extraction_id}")
if r3.status_code == 200:
    print(f"Status: {r3.status_code}, Windows: {r3.json()['total_windows']}")
else:
    print(f"Error: {r3.status_code}")


3. Get saved
Status: 200, Windows: 9


## Проверка эндпоинтов АВТОЭНКОДЕРА

In [7]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

df_common = pd.read_csv('data/window_features/optimized_features/df_common_features.csv')
df_bearing = pd.read_csv('data/window_features/optimized_features/df_bearing_features.csv')
df_eccentricity = pd.read_csv('data/window_features/optimized_features/df_eccentricity_features.csv')
df_rotor = pd.read_csv('data/window_features/optimized_features/df_rotor_features.csv')
df_stator = pd.read_csv('data/window_features/optimized_features/df_stator_features.csv')

meta_cols = ['file_name', 'file_index', 'window_index', 'window_start_sample', 
             'window_end_sample', 'time_start_sec', 'time_end_sec']

dfs = [df_common, df_bearing, df_eccentricity, df_rotor, df_stator]

for i, df in enumerate(dfs):
    drop_cols = [col for col in meta_cols if col in df.columns]
    if drop_cols:
        df = df.drop(columns=drop_cols)
        dfs[i] = df
    
    imputer = KNNImputer(n_neighbors=5)
    imputed_values = imputer.fit_transform(df)
    dfs[i] = pd.DataFrame(imputed_values, columns=df.columns)

df_common, df_bearing, df_eccentricity, df_rotor, df_stator = dfs

In [10]:
import numpy as np

N = 50

# ✅ Определяем минимальное количество строк среди всех датафреймов
min_rows = min(len(df_common), len(df_bearing), len(df_eccentricity), len(df_rotor), len(df_stator))

# ✅ Выбираем N случайных индексов (одни и те же для всех DF)
if N > min_rows:
    print(f"⚠️ Запрошено {N} строк, но доступно только {min_rows}. Берем все доступные.")
    random_indices = np.arange(min_rows)
    N = min_rows
else:
    random_indices = np.random.choice(min_rows, size=N, replace=False)  # Без повторов
    # Или если хотите с возможными повторами:
    # random_indices = np.random.choice(min_rows, size=N, replace=True)

print(f"📊 Выбранные случайные индексы: {sorted(random_indices)}")

# ✅ Используем одни и те же индексы для всех датафреймов
sample_data = pd.concat([
    df_common.iloc[random_indices].reset_index(drop=True),
    df_bearing.iloc[random_indices].reset_index(drop=True), 
    df_eccentricity.iloc[random_indices].reset_index(drop=True),
    df_rotor.iloc[random_indices].reset_index(drop=True),
    df_stator.iloc[random_indices].reset_index(drop=True),
], axis=1)

print(f"✅ Создан sample_data с {len(sample_data)} синхронизированными строками")
print(f"📏 Форма: {sample_data.shape}")


📊 Выбранные случайные индексы: [187, 693, 846, 898, 1372, 1842, 3983, 4446, 4885, 5457, 5500, 6049, 6292, 6683, 7163, 7276, 7697, 8024, 8726, 9774, 9780, 10217, 10623, 10913, 11294, 11412, 11449, 11818, 12164, 12273, 12349, 12463, 12698, 14030, 14338, 14728, 15041, 16059, 16242, 17213, 17273, 17649, 18419, 18724, 19217, 21346, 21383, 21539, 21741, 22853]
✅ Создан sample_data с 50 синхронизированными строками
📏 Форма: (50, 119)


In [11]:
import requests
import pandas as pd
import uuid

data = sample_data.values.tolist()

batch_id = str(uuid.uuid4())

payload = {
    "input": data,
    "batch_id": batch_id
}

url = "http://localhost:8000/autoencoder/batch_predict"
response = requests.post(url, json=payload)
response.raise_for_status()
results = response.json()

output_rows = []

for idx, out in enumerate(results['results']):
    row = {
        "idx": idx,
        "system_anomaly": out['overall']['system_is_anomaly'],
        "system_status": out['overall']['system_health_status'],
        "bearing_anomaly": out['bearing']['is_anomaly'],
        "rotor_anomaly": out['rotor']['is_anomaly'],
        "eccentricity_anomaly": out['eccentricity']['is_anomaly'],
        "stator_anomaly": out['stator']['is_anomaly'],
    }
    output_rows.append(row)

df_result = pd.DataFrame(output_rows)
print(f'"batch_id": {batch_id}')
display(df_result)


"batch_id": 876ddb4c-8e25-43ff-a781-f6d259224bf9


,idx,system_anomaly,system_status,bearing_anomaly,rotor_anomaly,eccentricity_anomaly,stator_anomaly
0,0,False,Healthy,False,False,False,False
1,1,False,Healthy,False,False,False,False
2,2,False,Healthy,False,False,False,False
3,3,False,Healthy,False,False,False,False
4,4,False,Healthy,False,False,False,False
5,5,False,Healthy,False,False,False,False
6,6,False,Healthy,False,False,False,False
7,7,False,Healthy,False,False,False,False
8,8,False,Healthy,False,False,False,False
9,9,False,Healthy,False,False,False,False


## Проверка эндпоинтов DUAL_LSTM

In [6]:
import requests
import pandas as pd
import uuid

BASE_URL = "http://127.0.0.1:8000"

test_sequence = np.random.random((10, 119)).tolist()
batch_id = str(uuid.uuid4())

payload = {
    "data": test_sequence,
    "n_steps": 5,
    "batch_id": batch_id
}

response = requests.post(f"{BASE_URL}/dual-lstm/predict", json=payload)
result = response.json()

inference_id = result['inference_id']

print(f"Inference ID: {inference_id}")

response = requests.get(f"{BASE_URL}/dual-lstm/results/{inference_id}")
stored_result = response.json()

print(f"Retrieved result status: {response.status_code}")
print(f"Predictions shape: {len(stored_result['predictions']['values'])} x {len(stored_result['predictions']['values'][0])}")

response = requests.get(f"{BASE_URL}/dual-lstm/batch/{batch_id}/results")
batch_results = response.json()

print(f"Batch results count: {batch_results['count']}")
print(f"Batch ID: {batch_results['batch_id']}")

predictions_from_db = pd.DataFrame(stored_result['predictions']['values'], columns=sample_data.columns)
predictions_from_db.index = [f'Step_{i+1}' for i in range(5)]

display(predictions_from_db)


Inference ID: 070daa2a-d9e8-43b5-bfd7-5e5737b85877
Retrieved result status: 200
Predictions shape: 5 x 119
Batch results count: 1
Batch ID: 2188f56d-2834-453d-8b6e-bc707dd2a55f


,rms_A,mean_A,std_A,rms_B,mean_B,std_B,rms_C,mean_C,std_C,total_imbalance,...,modulation_coeff_C,rel_energy_low_band_A,rel_energy_medium_band_A,rel_energy_high_band_A,rel_energy_low_band_B,rel_energy_medium_band_B,rel_energy_high_band_B,rel_energy_low_band_C,rel_energy_medium_band_C,rel_energy_high_band_C
Step_1,2.378441,0.037511,2.378121,2.369321,-0.073398,2.368138,1.857060,0.038332,1.856627,0.157789,...,0.068794,0.000780,0.992246,0.000369,0.001458,0.991828,0.000341,0.000923,0.991003,0.000284
Step_2,2.382078,0.037635,2.381753,2.371736,-0.071639,2.370605,1.859840,0.038944,1.859387,0.157819,...,0.070364,0.000796,0.992162,0.000383,0.001428,0.991780,0.000354,0.000947,0.990881,0.000300
Step_3,2.657746,0.037053,2.657466,2.643390,-0.068906,2.642424,2.071880,0.035862,2.071520,0.158888,...,0.075224,0.000634,0.991130,0.000719,0.001150,0.990971,0.000661,0.000728,0.989912,0.000658
Step_4,2.705370,0.036966,2.705098,2.690271,-0.068312,2.689336,2.108505,0.035357,2.108159,0.159111,...,0.076204,0.000608,0.990945,0.000778,0.001103,0.990853,0.000716,0.000694,0.989769,0.000721
Step_5,2.706136,0.036969,2.705863,2.691037,-0.068316,2.690102,2.109096,0.035344,2.108751,0.159111,...,0.076206,0.000607,0.990943,0.000779,0.001103,0.990850,0.000717,0.000693,0.989765,0.000722


## Проверка эндпоинтов Hybrid LSTM

In [7]:
import requests
import numpy as np
import pandas as pd
import uuid

BASE_URL = "http://127.0.0.1:8000"

def check_status(response, expected_keys=None):
    if response.status_code == 200:
        data = response.json()
        if expected_keys and all(key in data for key in expected_keys):
            return True, data
        elif not expected_keys:
            return True, data
    return False, None

print("🧪 Hybrid LSTM API Tests")
print("-" * 30)

success, data = check_status(requests.get(f"{BASE_URL}/hybrid-lstm/health"), ['status'])
print(f"{'✓' if success else '✗'} Health Check")

test_sequence = np.random.random((10, 187)).tolist()
batch_id = str(uuid.uuid4())

payload = {
    "data": test_sequence,
    "n_steps": 5,
    "batch_id": batch_id
}

success, result = check_status(requests.post(f"{BASE_URL}/hybrid-lstm/predict", json=payload), 
                              ['predictions', 'inference_id', 'status'])
print(f"{'✓' if success else '✗'} Prediction")

if success:
    inference_id = result['inference_id']
    
    success, _ = check_status(requests.get(f"{BASE_URL}/hybrid-lstm/results/{inference_id}"), 
                             ['predictions', 'metadata'])
    print(f"{'✓' if success else '✗'} Get Result")
    
    success, _ = check_status(requests.get(f"{BASE_URL}/hybrid-lstm/batch/{batch_id}/results"), 
                             ['batch_id', 'results'])
    print(f"{'✓' if success else '✗'} Batch Results")
    
    success, _ = check_status(requests.get(f"{BASE_URL}/hybrid-lstm/attention/{inference_id}"), 
                             ['attention_analysis'])
    print(f"{'✓' if success else '✗'} Attention Analysis")
    
    print("\n📊 Prediction Results:")
    predictions_df = pd.DataFrame(result['predictions']['values'])
    predictions_df.index = [f'Step_{i+1}' for i in range(5)]
    display(predictions_df.head())
    
else:
    print("✗ Tests failed - no predictions to show")


🧪 Hybrid LSTM API Tests
------------------------------
✓ Health Check
✓ Prediction
✓ Get Result
✓ Batch Results
✓ Attention Analysis

📊 Prediction Results:


,0,1,2,3,4,5,6,7,8,9,...,109,110,111,112,113,114,115,116,117,118
Step_1,2.420545,0.027857,2.420375,2.408508,-0.066625,2.407557,1.890549,0.038616,1.890129,0.134923,...,0.058654,0.000645,0.689423,0.000415,0.001148,0.588473,0.000330,0.000649,0.587743,0.000294
Step_2,2.408614,0.028398,2.408436,2.395993,-0.066771,2.395030,1.881158,0.038213,1.880745,0.133592,...,0.057728,0.000633,0.689489,0.000399,0.001143,0.582481,0.000315,0.000634,0.581763,0.000280
Step_3,2.393656,0.028537,2.393473,2.381365,-0.067399,2.380374,1.869672,0.038342,1.869254,0.132067,...,0.056576,0.000618,0.689568,0.000380,0.001144,0.576079,0.000296,0.000618,0.575372,0.000260
Step_4,2.366787,0.028666,2.366597,2.355218,-0.068600,2.354179,1.849093,0.038901,1.848660,0.130670,...,0.055308,0.000603,0.689689,0.000346,0.001152,0.570536,0.000271,0.000604,0.569845,0.000234
Step_5,2.338643,0.029093,2.338444,2.327290,-0.069815,2.326202,1.827369,0.039396,1.826922,0.128419,...,0.053749,0.000596,0.689796,0.000310,0.001151,0.560966,0.000244,0.000590,0.560288,0.000206


## Тестирование двигателя

In [33]:
# Ячейка 1: Импорты и настройки
import requests
import websocket
import json
import time
import threading
from IPython.display import display, HTML, clear_output

BASE_URL = "http://localhost:8005"
WS_URL = "ws://localhost:8005/ws"

print("🔧 Настройки готовы")
print(f"API URL: {BASE_URL}")
print(f"WebSocket URL: {WS_URL}")

# Ячейка 4: Тест POST /start
def test_start_motor():
    print("🟢 Тестирование Start Motor...")
    try:
        response = requests.post(f"{BASE_URL}/start")
        print(f"Status: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            print(f"✅ Мотор запущен: {data}")
            
            # Проверяем статус после запуска
            time.sleep(0.5)
            status_response = requests.get(f"{BASE_URL}/status")
            if status_response.status_code == 200:
                status_data = status_response.json()
                print(f"  Статус после запуска: running={status_data.get('running')}")
        else:
            print(f"❌ Ошибка запуска: {response.status_code}")
            
    except Exception as e:
        print(f"❌ Ошибка: {e}")

test_start_motor()



🔧 Настройки готовы
API URL: http://localhost:8005
WebSocket URL: ws://localhost:8005/ws
🟢 Тестирование Start Motor...
Status: 200
✅ Мотор запущен: {'status': 'Motor started', 'config': {'frequency': 50.0, 'sampling_rate': 25600.0, 'amplitude': 3.0, 'noise_level': 0.05, 'anomaly_chance': 0.0, 'phase_drop_chance': 0.0, 'voltage_imbalance': 0.02, 'output_batch_size': 1, 'csv_files': ['current_1.csv']}}
  Статус после запуска: running=True


In [37]:
response = requests.get(f"{BASE_URL}/motor")
response.json()

{'motor_type': 'Асинхронный электродвигатель 3 кВт',
 'speeds': {'motor': '1770 об/мин', 'output': '3010 об/мин', 'ratio': 1.7},
 'bearings': 'NSK6205DDU (опоры А и В)',
 'load_system': 'Электрический тормоз + муфты',
 'monitoring': {'sensors': 'Токовые датчики 3 фазы',
  'sampling_rate': '25600.0 Hz',
  'adc': 'Многоканальный модуль, 16-bit',
  'recording': 'Синхронная запись CSV'},
 'electrical': {'frequency': '50.0 Hz',
  'current': '3.0 A',
  'phases': 'R-S-T, 120°',
  'imbalance': '2.0%'},
 'data_status': {'source': 'CSV replay (current_1.csv + current_4.csv)',
  'samples': 1536492,
  'duration': '60.0 sec',
  'position': '1331707/1536492',
  'running': True},
 'bearing_frequencies_hz': {'bpfo': 158.7,
  'bpfi': 196.3,
  'bsf': 67.4,
  'ftf': 20.9},
 'quality': {'thd': '< 5%',
  'accuracy': '±0.1%',
  'frequency_range': 'DC - 12.8 kHz'}}

In [23]:
# Ячейка 7: Тест POST /stop
def test_stop_motor():
    print("🔴 Тестирование Stop Motor...")
    try:
        response = requests.post(f"{BASE_URL}/stop")
        print(f"Status: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            print(f"✅ Мотор остановлен: {data}")
            
            # Проверяем статус после остановки
            time.sleep(0.5)
            status_response = requests.get(f"{BASE_URL}/status")
            if status_response.status_code == 200:
                status_data = status_response.json()
                print(f"  Статус после остановки: running={status_data.get('running')}")
        else:
            print(f"❌ Ошибка остановки: {response.status_code}")
            
    except Exception as e:
        print(f"❌ Ошибка: {e}")

test_stop_motor()


🔴 Тестирование Stop Motor...
Status: 200
✅ Мотор остановлен: {'status': 'Motor stopped'}
  Статус после остановки: running=False


## Полный пайплайн системы
- получение данных от двигателя
- преобразование тока в окна
- прогнозирование  на 10 окон с помощью Dual_LSTM
- Классификация автоэнкодером
- прогнозирование  на 10 окон с помощью BiLSTM

In [23]:
# Простой тест для pipeline эндпоинта в Jupyter Notebook

import requests
import json
import numpy as np
from datetime import datetime

# Настройки
BASE_URL = "http://127.0.0.1:8000"  # Адрес вашего FastAPI сервера
PIPELINE_URL = f"{BASE_URL}/pipeline/analyze"

print("🔧 Тест пайплайна анализа данных двигателя")
print("=" * 50)

# 1. Генерация тестовых данных трёхфазного тока
print("📊 Генерация тестовых данных...")

# Создаем синтетические данные тока (например, 5000 точек)
sample_count = 70000
time = np.linspace(0, 1, sample_count)

# Моделируем трёхфазный ток с небольшими флуктуациями
current_R = 1.2 * np.sin(2 * np.pi * 50 * time) + 0.1 * np.random.normal(0, 1, sample_count)
current_S = 1.0 * np.sin(2 * np.pi * 50 * time - 2*np.pi/3) + 0.1 * np.random.normal(0, 1, sample_count) 
current_T = 1.1 * np.sin(2 * np.pi * 50 * time + 2*np.pi/3) + 0.1 * np.random.normal(0, 1, sample_count)

print(f"✅ Сгенерировано {sample_count} точек данных для каждой фазы")

# 2. Подготовка запроса
test_data = {
    "current_R": current_R.tolist(),
    "current_S": current_S.tolist(), 
    "current_T": current_T.tolist(),
    "user_id": "test_user_jupyter",
    "use_windowing": True,
    "window_size": 16384,
    "dual_lstm_steps": 5
}

print(f"📋 Данные подготовлены:")
print(f"   - Фаза R: {len(test_data['current_R'])} значений")
print(f"   - Фаза S: {len(test_data['current_S'])} значений") 
print(f"   - Фаза T: {len(test_data['current_T'])} значений")
print(f"   - Оконная обработка: {test_data['use_windowing']}")
print(f"   - Размер окна: {test_data['window_size']}")

# 3. Отправка запроса
print("\n🚀 Отправка запроса к пайплайну...")
start_time = datetime.now()

try:
    response = requests.post(
        PIPELINE_URL,
        json=test_data,
        headers={"Content-Type": "application/json"},
        timeout=300  # 5 минут таймаут
    )
    
    end_time = datetime.now()
    total_time = (end_time - start_time).total_seconds()
    
    print(f"⏱️  Общее время выполнения: {total_time:.2f} секунд")
    print(f"📡 HTTP статус: {response.status_code}")
    
    if response.status_code == 200:
        result = response.json()
        
        print("\n🎉 УСПЕХ! Пайплайн выполнен")
        print("=" * 50)
        
        # Общая информация
        print(f"🆔 Pipeline ID: {result['pipeline_id']}")
        print(f"👤 User ID: {result['user_id']}")  
        print(f"📅 Timestamp: {result['timestamp']}")
        print(f"⏰ Время выполнения: {result['total_execution_time_ms']:.2f} мс")
        print(f"📊 Общий статус: {result['overall_status'].upper()}")
        
        # Информация о данных
        print(f"\n📈 Обработанные данные:")
        data_summary = result['data_summary']
        print(f"   - Длина данных: {data_summary['data_length']}")
        print(f"   - Фазы: {', '.join(data_summary['phases'])}")
        print(f"   - Оконная обработка: {'Да' if data_summary['use_windowing'] else 'Нет'}")
        if data_summary['window_size']:
            print(f"   - Размер окна: {data_summary['window_size']}")
        
        # Детали по этапам
        print(f"\n🔍 Детали выполнения этапов:")
        for i, stage in enumerate(result['stages'], 1):
            status_emoji = "✅" if stage['success'] else "❌"
            print(f"   {i}. {status_emoji} {stage['stage'].replace('_', ' ').title()}")
            print(f"      - Статус: {stage['status']}")
            print(f"      - Время: {stage['execution_time_ms']:.2f} мс")
            if stage['result_id']:
                print(f"      - ID результата: {stage['result_id']}")
            if stage['error_message']:
                print(f"      - Ошибка: {stage['error_message']}")
            print()
        
        # Подсчет успешных этапов
        successful_stages = sum(1 for s in result['stages'] if s['success'])
        total_stages = len(result['stages'])
        print(f"📋 Итого: {successful_stages}/{total_stages} этапов выполнено успешно")
        
    else:
        print(f"❌ ОШИБКА! HTTP {response.status_code}")
        try:
            error_detail = response.json()
            print(f"📄 Детали ошибки: {error_detail}")
        except:
            print(f"📄 Ответ сервера: {response.text}")

except requests.exceptions.Timeout:
    print("⏰ ТАЙМАУТ! Запрос превысил лимит времени (5 минут)")
    
except requests.exceptions.ConnectionError:
    print("🔌 ОШИБКА ПОДКЛЮЧЕНИЯ! Проверьте:")
    print("   - Запущен ли FastAPI сервер?")
    print("   - Корректен ли адрес сервера?")
    print(f"   - URL: {PIPELINE_URL}")
    
except Exception as e:
    print(f"💥 НЕОЖИДАННАЯ ОШИБКА: {str(e)}")

print("\n" + "=" * 50)
print("🏁 Тест завершен")


🔧 Тест пайплайна анализа данных двигателя
📊 Генерация тестовых данных...
✅ Сгенерировано 70000 точек данных для каждой фазы
📋 Данные подготовлены:
   - Фаза R: 70000 значений
   - Фаза S: 70000 значений
   - Фаза T: 70000 значений
   - Оконная обработка: True
   - Размер окна: 16384

🚀 Отправка запроса к пайплайну...
⏱️  Общее время выполнения: 8.38 секунд
📡 HTTP статус: 200

🎉 УСПЕХ! Пайплайн выполнен
🆔 Pipeline ID: b687d12b-acb3-4ee0-9cf2-c1a61d092e4c
👤 User ID: test_user_jupyter
📅 Timestamp: 2025-08-11T14:08:18.700306
⏰ Время выполнения: 8200.05 мс
📊 Общий статус: SUCCESS

📈 Обработанные данные:
   - Длина данных: 70000
   - Фазы: R, S, T
   - Оконная обработка: Да
   - Размер окна: 16384

🔍 Детали выполнения этапов:
   1. ✅ Feature Extraction
      - Статус: completed
      - Время: 521.19 мс
💥 НЕОЖИДАННАЯ ОШИБКА: 'result_id'

🏁 Тест завершен


In [14]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime

BASE_URL = "http://127.0.0.1:8000"

def extract_clean_features(window):
    feature_vector = []
    expected_groups = ['bearing', 'common', 'eccentricity', 'rotor', 'stator']
    excluded_keys = {'window_index', 'window_start_sample', 'window_end_sample', 'window_metadata', 'batch_id', 'user_id', 'timestamp'}
    group_counts = {}
    
    for group in expected_groups:
        if group in window:
            group_data = window[group]
            group_features = []
            if isinstance(group_data, dict):
                for key, value in group_data.items():
                    if (isinstance(value, (int, float)) and key not in excluded_keys and 
                        not key.startswith('window_') and not key.startswith('meta_')):
                        group_features.append(value)
            feature_vector.extend(group_features)
            group_counts[group] = len(group_features)
    
    return feature_vector, group_counts

print("📁 Loading first 70k rows from current_1.csv...")

df = pd.read_csv('data/time_series/current_1.csv', nrows=70000)
print(f"✅ Loaded CSV shape: {df.shape}, columns: {list(df.columns)}")

current_columns = {}
possible_names = {
    'R': ['current_R', 'current_r', 'R', 'phase_R', 'phase_a', 'A'],
    'S': ['current_S', 'current_s', 'S', 'phase_S', 'phase_b', 'B'], 
    'T': ['current_T', 'current_t', 'T', 'phase_T', 'phase_c', 'C']
}

for phase, candidates in possible_names.items():
    for candidate in candidates:
        if candidate in df.columns:
            current_columns[phase] = candidate
            break

if len(current_columns) != 3:
    numeric_cols = df.select_dtypes(include=[np.number]).columns[:3]
    current_columns = {'R': numeric_cols[0], 'S': numeric_cols[1], 'T': numeric_cols[2]}

current_R = df[current_columns['R']].values
current_S = df[current_columns['S']].values  
current_T = df[current_columns['T']].values

print(f"📊 Data: R={len(current_R)}, range=[{current_R.min():.3f}, {current_R.max():.3f}]")

test_batch_id = f"debug_batch_{int(datetime.now().timestamp())}"
test_data = {
    "current_R": current_R.tolist(),
    "current_S": current_S.tolist(), 
    "current_T": current_T.tolist(),
    "user_id": "debug_user",
    "batch_id": test_batch_id,
    "use_windowing": True,
    "window_size": 16384,
    "dual_lstm_steps": 5
}

print(f"🎯 Testing batch_id: {test_batch_id}")
response = requests.post(f"{BASE_URL}/pipeline/analyze", json=test_data, timeout=300)

if response.status_code == 200:
    result = response.json()
    returned_batch_id = result['batch_id']
    print(f"✅ Pipeline: {result.get('overall_status')}, Batch ID: {returned_batch_id}")
    
    features_response = requests.get(f"{BASE_URL}/features/batch/{returned_batch_id}/results")
    
    if features_response.status_code == 200:
        features_data = features_response.json()
        windows = features_data['features']['windows']
        
        print(f"\n🔍 FIRST WINDOW DEBUG:")
        if len(windows) > 0:
            first_window = windows[0]
            print(f"Keys: {list(first_window.keys())}")
            
            for group in ['bearing', 'common', 'eccentricity', 'rotor', 'stator']:
                if group in first_window:
                    group_data = first_window[group]
                    if isinstance(group_data, dict):
                        numeric_values = [v for k, v in group_data.items() if isinstance(v, (int, float)) and not k.startswith('window_')]
                        if numeric_values:
                            print(f"{group}: {len(numeric_values)} values, range [{min(numeric_values):.6f}, {max(numeric_values):.6f}]")
                            print(f"  First 3: {numeric_values[:3]}")
                            print(f"  Keys: {[k for k, v in group_data.items() if isinstance(v, (int, float)) and not k.startswith('window_')][:3]}")
        
        feature_vectors = []
        for i, window in enumerate(windows):
            clean_features, _ = extract_clean_features(window)
            feature_vectors.append([i] + clean_features)
        
        if feature_vectors:
            feature_array = np.array([fv[1:] for fv in feature_vectors])
            print(f"\n🔍 FEATURE MATRIX STATS:")
            print(f"Shape: {feature_array.shape}")
            print(f"Range: [{feature_array.min():.6f}, {feature_array.max():.6f}]")
            print(f"Mean: {feature_array.mean():.6f}")
            print(f"Std: {feature_array.std():.6f}")
            print(f"First row sample: {feature_array[0, :5].tolist()}")
        
        df_features = pd.DataFrame(feature_vectors, columns=['window_index'] + [f'f_{j:03d}' for j in range(len(feature_vectors[0])-1)])
        print(f"\n📊 FEATURES: {df_features.shape}")
        display(df_features.head(3))
    
    autoencoder_response = requests.get(f"{BASE_URL}/autoencoder/batch/{returned_batch_id}/results")
    
    if autoencoder_response.status_code == 200:
        autoencoder_data = autoencoder_response.json()
        results = autoencoder_data['results']
        
        errors = [r.get('overall', {}).get('overall_reconstruction_error', 0) for r in results]
        anomalies = [r.get('overall', {}).get('system_is_anomaly', False) for r in results]
        
        print(f"\n🤖 AUTOENCODER: {len(results)} results")
        print(f"Anomaly rate: {sum(anomalies)}/{len(anomalies)} ({sum(anomalies)/len(anomalies)*100:.1f}%)")
        print(f"Error range: [{min(errors):.2e}, {max(errors):.2e}]")
        print(f"Error mean: {np.mean(errors):.2e}")
        
        df_auto = pd.DataFrame([{
            'window': i, 
            'anomaly': r.get('overall', {}).get('system_is_anomaly'),
            'error': r.get('overall', {}).get('overall_reconstruction_error'),
            'confidence': r.get('overall', {}).get('overall_confidence_score')
        } for i, r in enumerate(results)])
        
        display(df_auto.head())
    
    print(f"\n🎯 SUMMARY: {returned_batch_id}")
    if 'df_features' in locals():
        print(f"Features: {df_features.shape[1]-1} extracted")
    if 'errors' in locals():
        print(f"Autoencoder: {sum(anomalies)} anomalies, avg error {np.mean(errors):.2e}")

else:
    print(f"❌ Pipeline failed: {response.status_code}")
    print(response.text)


📁 Loading first 70k rows from current_1.csv...
✅ Loaded CSV shape: (70000, 3), columns: ['current_R', 'current_S', 'current_T']
📊 Data: R=70000, range=[-4.453, 4.501]
🎯 Testing batch_id: debug_batch_1755079329
✅ Pipeline: success, Batch ID: debug_batch_1755079329

🔍 FIRST WINDOW DEBUG:
Keys: ['common', 'rotor', 'stator', 'bearing', 'eccentricity', 'window_metadata']
bearing: 30 values, range [-0.187572, 31.978353]
  First 3: [3.7156107542433, 3.2798869446752836, 6.307011704469371]
  Keys: ['bearing_bpfo_amp_A', 'bearing_bpfi_amp_A', 'bearing_bpfo_amp_B']
common: 17 values, range [-0.086951, 3.039523]
  First 3: [2.3190205364905667, 0.03932191017944327, 2.318687136300375]
  Keys: ['rms_A', 'mean_A', 'std_A']
eccentricity: 29 values, range [-0.519180, 2884.399986]
  First 3: [0.19230006794660534, 0.1570121197403107, 0.012326438710756346]
  Keys: ['ecc_current_asymmetry', 'ecc_max_deviation', 'ecc_rms_variance']
rotor: 15 values, range [0.020704, 13349.742102]
  First 3: [1.82694352237909

,window_index,f_000,f_001,f_002,f_003,f_004,f_005,f_006,f_007,f_008,...,f_109,f_110,f_111,f_112,f_113,f_114,f_115,f_116,f_117,f_118
0,0,3.715611,3.279887,6.307012,4.424989,9.656678,4.543077,13.502977,7.265583,6.048716,...,0.056829,0.000467,0.993944,0.000229,0.001518,0.993536,0.000201,0.000464,0.993404,0.000135
1,1,3.525059,7.463655,6.397382,5.239218,8.441753,4.376282,11.882971,20.410128,5.080111,...,0.057138,0.000451,0.994084,0.000224,0.001584,0.993436,0.000212,0.000466,0.993433,0.000144
2,2,4.717289,5.404116,12.675927,2.077045,9.268340,2.767698,17.409517,7.836209,3.132467,...,0.056860,0.000396,0.994117,0.000240,0.001632,0.993327,0.000212,0.000516,0.993350,0.000158



🤖 AUTOENCODER: 14 results
Anomaly rate: 0/14 (0.0%)
Error range: [4.22e-01, 6.89e-01]
Error mean: 5.32e-01


,window,anomaly,error,confidence
0,0,False,0.430478,0.6072
1,1,False,0.535506,0.5810
2,2,False,0.421541,0.6492
3,3,False,0.547212,0.5913
4,4,False,0.616453,0.7295



🎯 SUMMARY: debug_batch_1755079329
Features: 119 extracted
Autoencoder: 0 anomalies, avg error 5.32e-01


## Тест стриминга

In [31]:
import requests
import time
import pandas as pd

BASE_URL = "http://127.0.0.1:8000"
USER_ID = "test_streaming_user"
response = requests.post(f"{BASE_URL}/streaming/pipeline/start/{USER_ID}")
result = response.json()

print(f"Start: {result['status']}")
print(f"Session: {result.get('session_id', 'N/A')}")


Start: already_running
Session: N/A


In [12]:
response = requests.get(f"{BASE_URL}/streaming/pipeline/status")
all_status = response.json()
print(f"Active streams: {all_status['active_pipeline_streams']}")

if all_status['streams']:
    df_streams = pd.DataFrame([
        {
            'user_id': user_id,
            'uptime_min': status['uptime_seconds']/60 if status['uptime_seconds'] else 0,
            'batches': status['processed_batches'],
            'progress': status['progress_percent'],
            'healthy': status['pipeline_stats']['healthy_batches'],
            'anomalies': status['pipeline_stats']['anomalies_detected']
        }
        for user_id, status in all_status['streams'].items()
    ])
    display(df_streams)


Active streams: 0


In [28]:
response = requests.post(f"{BASE_URL}/streaming/pipeline/stop/{USER_ID}")
result = response.json()
print(f"Stop: {result['status']}")
if 'pipeline_stats' in result:
    stats = result['pipeline_stats']
    print(f"Final stats:")
    print(f"  Duration: {result.get('duration_seconds', 0):.1f}s")
    print(f"  Batches processed: {result['processed_batches']}")
    print(f"  Features extracted: {stats['total_features']}")
    print(f"  Autoencoder runs: {stats['total_autoencoder']}")
    print(f"  LSTM predictions: {stats['total_lstm']}")
    print(f"  Health ratio: {stats['healthy_batches']}/{stats['healthy_batches'] + stats['anomalies_detected']}")


Stop: stopped
Final stats:
  Duration: 3754.6s
  Batches processed: 139
  Features extracted: 2780
  Autoencoder runs: 2780
  LSTM predictions: 1529
  Health ratio: 139/139
